In [1]:
# let the notebook access the code folder
import sys
sys.path.insert(1,"code")

We extend Cuppa1 with a 'declare' statement so that we have precise control in which scope the variable is being declared.

```python
# %load code/cuppa2_gram.py
# grammar for Cuppa2

from ply import yacc
from cuppa2_lex import tokens, lexer

# set precedence and associativity
# NOTE: all arithmetic operator need to have tokens
#       so that we can put them into the precedence table
precedence = (
              ('left', 'EQ', 'LE'),
              ('left', 'PLUS', 'MINUS'),
              ('left', 'TIMES', 'DIVIDE'),
              ('right', 'UMINUS', 'NOT')
             )


def p_grammar(_):
    '''
    program : stmt_list

    stmt_list : stmt stmt_list
              | empty

    stmt : DECLARE ID opt_init opt_semi
         | ID '=' exp opt_semi
         | GET ID opt_semi
         | PUT exp opt_semi
         | WHILE '(' exp ')' stmt
         | IF '(' exp ')' stmt opt_else
         | '{' stmt_list '}'

    opt_init : '=' exp
             | empty
             
    opt_else : ELSE stmt
             | empty
             
    opt_semi : ';'
             | empty

    exp : exp PLUS exp
        | exp MINUS exp
        | exp TIMES exp
        | exp DIVIDE exp
        | exp EQ exp
        | exp LE exp
        | INTEGER
        | ID
        | '(' exp ')'
        | MINUS exp %prec UMINUS
        | NOT exp
    '''
    pass

def p_empty(p):
    'empty :'
    pass

def p_error(t):
    print("Syntax error at '%s'" % t.value)

### build the parser
parser = yacc.yacc()
```

Here is the lexer for Cuppa2, it is the same lexer as for Cuppa1 except that this new lexer supports the keyword 'declare'.

```python
# %load code/cuppa2_lex.py
# Lexer for Cuppa2

from ply import lex

reserved = {
    'get'     : 'GET',
    'put'     : 'PUT',
    'if'      : 'IF',
    'else'    : 'ELSE',
    'while'   : 'WHILE',
    'not'     : 'NOT',
    'declare' : 'DECLARE'
}

literals = [';','=','(',')','{','}']

tokens = [
          'PLUS','MINUS','TIMES','DIVIDE',
          'EQ','LE', 
          'INTEGER','ID',
          ] + list(reserved.values())

t_PLUS    = r'\+'
t_MINUS   = r'-'
t_TIMES   = r'\*'
t_DIVIDE  = r'/'
t_EQ      = r'=='
t_LE      = r'<='

t_ignore = ' \t'

def t_ID(t):
    r'[a-zA-Z_][a-zA-Z_0-9]*'
    t.type = reserved.get(t.value,'ID')    # Check for reserved words
    return t

def t_INTEGER(t):
    r'[0-9]+'
    return t

def t_COMMENT(t):
    r'//.*'
    pass

def t_NEWLINE(t):
    r'\n'
    pass

def t_error(t):
    print("Illegal character %s" % t.value[0])
    t.lexer.skip(1)

# build the lexer
lexer = lex.lex(debug=0)
```


Testing the grammar/parser for Cuppa2.

In [2]:
from cuppa2_lex import lexer
from cuppa2_gram import parser

In [3]:
parser.parse("declare x = 1; put x", lexer=lexer)

# Cuppa2 Interpreter

The [frontend](code/cuppa2_frontend_gram.py) constructs the AST.  Note that we no longer just put symbols into the symbol as we discover them.  We have to delay that until later when we know more about scope.

The [tree walker](code/cuppa2_interp_walk.py) walks the AST and interprets the values.

We have our top-level interp function.

```python
# %load code/cuppa2_interp.py
#!/usr/bin/env python
# Cuppa2 interpreter

from argparse import ArgumentParser
from cuppa2_lex import lexer
from cuppa2_frontend_gram import parser
from cuppa2_state import state
from cuppa2_interp_walk import walk

def interp(input_stream):

    # initialize the state object
    state.initialize()

    # build the AST
    parser.parse(input_stream, lexer=lexer)

    # walk the AST
    walk(state.AST)

if __name__ == "__main__":
    # parse command line args
    aparser = ArgumentParser()
    aparser.add_argument('input')

    args = vars(aparser.parse_args())

    f = open(args['input'], 'r')
    input_stream = f.read()
    f.close()

    # execute interpreter
    interp(input_stream=input_stream)
```

So far this is almost identical to the interpreter for Cuppa1.  The big difference between the two languages is the structure of the symbol table.  In Cuppa1 the symbol table consisted just of a single global dictionary.  In Cuppa2 we need to keep track of scope.  Therefore the symbol in Cuppa2 consists of a stack of dictionaries: one dictionary per active scope.

In [4]:
# %load code/cuppa2_symtab.py
#########################################################################
# symbol table for Cuppa2
#
# it is a scoped symbol table with a dictionary at each scope level
#
#########################################################################

CURR_SCOPE = 0

class SymTab:

    #-------
    def __init__(self):
        # global scope dictionary must always be present
        self.scoped_symtab = [{}]

    #-------
    def push_scope(self):
        # push a new dictionary onto the stack - stack grows to the left
        self.scoped_symtab.insert(CURR_SCOPE,{})

    #-------
    def pop_scope(self):
        # pop the left most dictionary off the stack
        if len(self.scoped_symtab) == 1:
            raise ValueError("cannot pop the global scope")
        else:
            self.scoped_symtab.pop(CURR_SCOPE)

    #-------
    def declare_sym(self, sym, init):
        # declare the symbol in the current scope: dict @ position 0
        
        # first we need to check whether the symbol was already declared
        # at this scope
        if sym in self.scoped_symtab[CURR_SCOPE]:
            raise ValueError("symbol {} already declared".format(sym))
        
        # enter the symbol in the current scope
        scope_dict = self.scoped_symtab[CURR_SCOPE]
        scope_dict[sym] = init

    #-------
    def lookup_sym(self, sym):
        # find the first occurence of sym in the symtab stack
        # and return the associated value

        n_scopes = len(self.scoped_symtab)

        for scope in range(n_scopes):
            if sym in self.scoped_symtab[scope]:
                val = self.scoped_symtab[scope].get(sym)
                return val

        # not found
        raise ValueError("{} was not declared".format(sym))

    #-------
    def update_sym(self, sym, val):
        # find the first occurence of sym in the symtab stack
        # and update the associated value

        n_scopes = len(self.scoped_symtab)

        for scope in range(n_scopes):
            if sym in self.scoped_symtab[scope]:
                scope_dict = self.scoped_symtab[scope]
                scope_dict[sym] = val
                return

        # not found
        raise ValueError("{} was not declared".format(sym))

#########################################################################




Our test programs:

In [5]:
# %load code/cuppa2_examples.py
fact =\
'''
// computes the factorial of x
declare x;
declare y = 1;

get x;

while (1 <= x)
{
    y = y * x;
    x = x - 1;
}

put y;
'''

fold =\
'''
declare x = (3 + 2) / 5;
put x;
'''

if_ex =\
'''
declare y = 1

if (y == 1)
{
    put 1
}
else 
{
    put 2
}
'''

list =\
'''
// list of integers
declare x;
get x;
while (1 <= x)
{
    put x;
    x = x - 1;
}
'''

redecl =\
'''
declare x;
get x;
put x + 1;
declare x = 10;
put x;
'''

scope1 =\
'''
declare x = 1;
{
    declare x = 2;
    put x;
}
{
    declare x = 3;
    put x;
}
put x;
'''

scope2 =\
'''
declare x = 1;
put x;
{
    x = 2;
}
put x;
{
    x = 3;
}
put x;
'''

undecl =\
'''
get x;
put x + 1;
'''




In [6]:
from cuppa2_interp import interp
from cuppa2_examples import *

In [7]:
interp(fold)

> 1


In [8]:
interp(fact)


Value for x? 3
> 6


In [9]:
interp("declare x = 10")

In [10]:
interp(scope2)

> 1
> 2
> 3


In [11]:
interp(scope1)

> 2
> 3
> 1


# Cuppa2 Compiler

testing the Cuppa2 compiler

In [26]:
from cuppa2_examples import scope1, scope2
from cuppa2_cc import cc

In [21]:
print(cc("declare x = 7; put x"))

	store R$x 7 ;
	print R$x ;
	stop ;



In [27]:
print(scope1)


declare x = 1;
{
    declare x = 2;
    put x;
}
{
    declare x = 3;
    put x;
}
put x;



In [28]:
print(cc(scope1))

	store R$x 1 ;
	store R$$x 2 ;
	print R$$x ;
	store R$$x 3 ;
	print R$$x ;
	print R$x ;
	stop ;



In [29]:
print(scope2)


declare x = 1;
put x;
{
    x = 2;
}
put x;
{
    x = 3;
}
put x;



In [30]:
print(cc(scope2))

	store R$x 1 ;
	print R$x ;
	store R$x 2 ;
	print R$x ;
	store R$x 3 ;
	print R$x ;
	stop ;

